In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

import atac_utils

In [ ]:
outfigdir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures/250324'
!mkdir -p $outfigdir

# load data

In [ ]:
scores_abc = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/scores_l23abc.csv", 
                         index_col=0,
                        )
scores_abc['scores_c-a'] = scores_abc['scores_c'] - scores_abc['scores_a']
scores_abc

In [ ]:
 # cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adata.obs.index.values])
# # adata.obs.index = cells_rna
# adata

adata = anndata.read("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad")
adata
# %%time


# %%time
# f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
# adata = sc.read(f_rna)
# adata#

In [ ]:
adata.X = adata.raw.X

In [ ]:
adata.obs['scores_a'] = scores_abc.loc[adata.obs.index,'scores_a'].copy()
adata.obs['scores_b'] = scores_abc.loc[adata.obs.index,'scores_b'].copy()
adata.obs['scores_c'] = scores_abc.loc[adata.obs.index,'scores_c'].copy()
adata.obs['scores_c-a'] = scores_abc.loc[adata.obs.index,'scores_c-a'].copy()

# fix cond incompatibility # P6NR vs P6
adata.obs['cond'] = adata.obs['cond'].apply(lambda x: x.replace('NR', '')).copy()

In [ ]:
adata.obs['cond']

In [ ]:
sample_labels = adata.obs['Sample'].values
time_labels = [s[:-1].replace('DR', '') for s in sample_labels]

adata.obs['sample'] = sample_labels #
adata.obs['time']   = time_labels

uniq_samples = natsorted(np.unique(sample_labels))
nr_samples = [s for s in uniq_samples if "DR" not in s]
dr_samples = [s for s in uniq_samples if "DR" in s]

uniq_conds = np.array(natsorted(np.unique(adata.obs['cond'].values)))
print(uniq_conds)

In [ ]:
condcode2cond = atac_utils.CONDCODE_TO_COND
condcode2cond

In [ ]:
sample_conditions = np.array(list(condcode2cond.values()))
n_cond = len(sample_conditions)

nr_condcodes  = np.array([0,1,2,3,5,7,9])
dr_condcodes  = np.array([4,6,8,10])

nr_conditions = sample_conditions[nr_condcodes]
dr_conditions = sample_conditions[dr_condcodes]

nr_times = [6,8,10,12,14,17,21]
dr_times =        [12,14,17,21]
times = nr_times

print(sample_conditions)
print(nr_condcodes, nr_conditions)
print(dr_condcodes, dr_conditions)

In [ ]:
%%time
adatas_pk = []
for cond in sample_conditions:
    print(cond)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/pmat_l23concensus_{cond}.h5ad'
    adata_pk = sc.read(f)
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
n_cond = len(sample_conditions)
n_peak = adatas_pk[0].shape[1]
n_type = 5

# check results and stats

In [ ]:
def mean_bin(x, y, bins=None):
    df = pd.DataFrame() # 
    df['x'] = x
    df['y'] = y 
    
    if bins is None:
        binned, bins = pd.qcut(df['x'], 10, retbins=True)
        df['xbin'] = binned
    
    df_mean = df.groupby('xbin').mean()
    mid_bins = np.mean(np.vstack([bins[:-1], bins[1:]]).T, axis=1)
    
    df_mean['mid'] = mid_bins
    return df_mean

In [ ]:
%%time
fin1 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_qs_250411.npy'
fin2 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_l2fc_250411.npy'


In [ ]:
qs_tensor = np.load(fin1)
l2fc_tensor = np.load(fin2)

l2fc_th = np.log2(2)
l2fc_th_s = np.log2(1.2)
alpha_th = 0.05

In [ ]:
qs_ca   = qs_tensor[:,0,:]
qs_ba   = qs_tensor[:,1,:]
qs_bc   = qs_tensor[:,2,:]

l2fc_ca = l2fc_tensor[:,0,:]
l2fc_ba = l2fc_tensor[:,1,:]
l2fc_bc = l2fc_tensor[:,2,:]

In [ ]:
qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


l2fc_as = np.min([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_cs = np.min([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_bs = np.min([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th], axis=0)
cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th], axis=0)
# cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th, l2fc_as > l2fc_th_s], axis=0)
# cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th, l2fc_cs > l2fc_th_s], axis=0)
cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th, l2fc_bs > l2fc_th_s], axis=0)


instances, counts_a = np.unique(cond_sig_a.sum(axis=0), return_counts=True)
instances, counts_c = np.unique(cond_sig_c.sum(axis=0), return_counts=True)
instances, counts_b = np.unique(cond_sig_b.sum(axis=0), return_counts=True)

print('num A genes for each cond:\t', cond_sig_a.sum(axis=1), np.any(cond_sig_a, axis=0).sum())
print('num C genes for each cond:\t', cond_sig_c.sum(axis=1), np.any(cond_sig_c, axis=0).sum())
print('num B genes for each cond:\t', cond_sig_b.sum(axis=1), np.any(cond_sig_b, axis=0).sum())

print('num A genes in num conds:\t',  counts_a[1:])
print('num C genes in num conds:\t',  counts_c[1:])
print('num B genes in num conds:\t',  counts_b[1:])

In [ ]:
%%time

all_x = []
all_y_a = []
all_y_c = []
all_y_b = []
all_nums = []

offset = 1

for cond_order, cond in condcode2cond.items():
    # get sub
    adatasub = adata[adata.obs['cond']==cond]
    cells_rna = adatasub.obs.index.values
    
    # get ATAC
    adata_pk = adatas_pk[cond_order]
    cells_atac = adata_pk.obs.index.values
    
    cells_mult = np.intersect1d(cells_rna, cells_atac)
    
    # no normalization here
    # x = adatasub[cells_mult].obsm['pca_p17on'][...,0]
    x = adatasub[cells_mult].obs['scores_c-a'].values
    
    cellcov = np.array(adata_pk[cells_mult].X.sum(axis=1)) #.reshape(-1,)
    
    mat_a = np.array(adata_pk[cells_mult, cond_sig_a[cond_order]].X.todense()) 
    mat_b = np.array(adata_pk[cells_mult, cond_sig_b[cond_order]].X.todense()) 
    mat_c = np.array(adata_pk[cells_mult, cond_sig_c[cond_order]].X.todense()) 
    
    # size norm (CPM) between cells
    mat_a = np.log2(mat_a/cellcov*1e6 + 1)
    mat_b = np.log2(mat_b/cellcov*1e6 + 1)
    mat_c = np.log2(mat_c/cellcov*1e6 + 1)
    
    y_a = np.mean(mat_a, axis=1)
    y_b = np.mean(mat_b, axis=1) 
    y_c = np.mean(mat_c, axis=1)
    
    
    all_x.append(x)
    all_y_a.append(y_a)
    all_y_c.append(y_c)
    all_y_b.append(y_b)
    all_nums.append([cond_sig_a[cond_order].sum(),
                     cond_sig_b[cond_order].sum(),
                     cond_sig_c[cond_order].sum(),
                    ])
    

In [ ]:
nx = len(nr_conditions)
fig, axs = plt.subplots(3,nx, figsize=(4*nx,4*3)) 
for i, cond_code in enumerate(nr_condcodes):
    cond = atac_utils.decode_cond(cond_code)
    x   = all_x[cond_code]
    y_a = all_y_a[cond_code]
    y_c = all_y_c[cond_code]
    y_b = all_y_b[cond_code]
    num_a, num_b, num_c = all_nums[cond_code]
    
    ax = axs[0][i]
    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_a, p  = stats.spearmanr(x, y_a)
    means = mean_bin(x, y_a)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'{cond}\nNa={num_a:,}, r={r_a:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)

    ax = axs[1][i]
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_c, p  = stats.spearmanr(x, y_c)
    means = mean_bin(x, y_c)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nc={num_c:,}, r={r_c:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    ax = axs[2][i]
    ax.scatter(x, y_b, s=1, color='C1', label='B peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_b, p  = stats.spearmanr(x, y_b)
    means = mean_bin(x, y_b)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nb={num_b:,}, r={r_b:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    
    if np.all(np.isnan(y_b)):
        ax.set_xticks([])
        ax.set_yticks([])
    
axs[0][0].set_xlabel('RNA PC1 (P17-21)')
axs[0][0].set_ylabel('ATAC peak accessibility')
fig.tight_layout()
plt.show()

In [ ]:
nx = len(nr_conditions)
fig, axs = plt.subplots(3,nx, figsize=(4*nx,4*3), sharey=True) 
for i, cond_code in enumerate(nr_condcodes):
    cond = atac_utils.decode_cond(cond_code)
    x   = all_x[cond_code]
    y_a = all_y_a[cond_code]
    y_c = all_y_c[cond_code]
    y_b = all_y_b[cond_code]
    num_a, num_b, num_c = all_nums[cond_code]
    
    ax = axs[0][i]
    # ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
    # r_a, p  = stats.spearmanr(x, y_a)
    means = mean_bin(x, y_a)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'{cond}\nNa={num_a:,}, r={r_a:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)

    ax = axs[1][i]
    # ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')
    # r_c, p  = stats.spearmanr(x, y_c)
    means = mean_bin(x, y_c)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nc={num_c:,}, r={r_c:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    ax = axs[2][i]
    # ax.scatter(x, y_b, s=1, color='C1', label='B peaks', alpha=0.5)# , facecolor='none')
    # r_b, p  = stats.spearmanr(x, y_b)
    means = mean_bin(x, y_b)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nb={num_b:,}, r={r_b:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    
    if np.all(np.isnan(y_b)):
        ax.set_xticks([])
        ax.set_yticks([])
    
axs[0][0].set_xlabel('RNA PC1 (P17-21)')
axs[0][0].set_ylabel('ATAC peak accessibility')
fig.tight_layout()
plt.show()

In [ ]:
nx = len(times)
fig, axs = plt.subplots(6,nx, figsize=(4*nx,4*6)) 
for i, cond_code in enumerate(nr_condcodes):
    cond = atac_utils.decode_cond(cond_code)
    x   = all_x[cond_code]
    y_a = all_y_a[cond_code]
    y_c = all_y_c[cond_code]
    y_b = all_y_b[cond_code]
    num_a, num_b, num_c = all_nums[cond_code]
    
    ax = axs[0*2][i]
    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
    r_a, p  = stats.spearmanr(x, y_a)
    means = mean_bin(x, y_a)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'{cond}\nNa={num_a:,}, r={r_a:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)

    ax = axs[1*2][i]
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')
    r_c, p  = stats.spearmanr(x, y_c)
    means = mean_bin(x, y_c)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nc={num_c:,}, r={r_c:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    ax = axs[2*2][i]
    ax.scatter(x, y_b, s=1, color='C1', label='B peaks', alpha=0.5)# , facecolor='none')
    r_b, p  = stats.spearmanr(x, y_b)
    means = mean_bin(x, y_b)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nb={num_b:,}, r={r_b:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    
for i, cond_code in enumerate(dr_condcodes):
    cond = atac_utils.decode_cond(cond_code)
    x   = all_x[cond_code]
    y_a = all_y_a[cond_code]
    y_c = all_y_c[cond_code]
    y_b = all_y_b[cond_code]
    num_a, num_b, num_c = all_nums[cond_code]
    
    ax = axs[0*2+1][i+3]
    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_a, p  = stats.spearmanr(x, y_a)
    means = mean_bin(x, y_a)
    ax.plot(means['x'], means['y'], color='k',)# , facecolor='none')
    ax.set_title(f'{cond}\nNa={num_a:,}, r={r_a:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)

    ax = axs[1*2+1][i+3]
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_c, p  = stats.spearmanr(x, y_c)
    means = mean_bin(x, y_c)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nc={num_c:,}, r={r_c:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    ax = axs[2*2+1][i+3]
    ax.scatter(x, y_b, s=1, color='C1', label='B peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r_b, p  = stats.spearmanr(x, y_b)
    means = mean_bin(x, y_b)
    ax.plot(means['x'], means['y'], color='k', )# , facecolor='none')
    ax.set_title(f'Nb={num_b:,}, r={r_b:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)

for ax in axs[1][:3]:
    ax.axis('off')
for ax in axs[3][:3]:
    ax.axis('off')
for ax in axs[5][:3]:
    ax.axis('off')
    
axs[0][0].set_xlabel('RNA PC1 (P17-21)')
axs[0][0].set_ylabel('ATAC peak accessibility')
fig.tight_layout()
output = os.path.join(outfigdir, 'atac_abc_celllevel.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
nr_condcodes

In [ ]:

cond_code = nr_condcodes[-1]
cond = atac_utils.decode_cond(cond_code)
x   = all_x[cond_code]
y_a = all_y_a[cond_code]
y_b = all_y_b[cond_code]
y_c = all_y_c[cond_code]
num_a, num_b, num_c = all_nums[cond_code]

tags = ['A', 'B', 'C']
nums = [num_a, num_b, num_c]
ys = [y_a, y_b, y_c]

fig, axs = plt.subplots(1,3, figsize=(4*3,5*1)) 
for i in range(3):
    ax = axs[i]
    num = nums[i]
    tag = tags[i]
    y = ys[i]
    
    ax.scatter(y, x, s=1, color=f'C{i}', label=f'{tag} peaks', alpha=0.5, rasterized=True)# , facecolor='none')
    r, p  = stats.spearmanr(x, y)
    means = mean_bin(x, y)
    ax.plot(means['y'], means['x'], color='k', rasterized=True)# , facecolor='none')
    ax.set_title(f'{cond}\nN={num:,}, r={r:.2f}')
    sns.despine(ax=ax)
    ax.grid(False)
    
    ax.invert_yaxis()

    
axs[0].set_xlabel('Chromatin accessibility')
axs[0].set_ylabel('A-C score')
fig.tight_layout()
output = os.path.join(outfigdir, 'atac_abc_celllevel_p21.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(1*4,1*4)) # , sharex=True)

num_uniq_a = np.sum(np.any(cond_sig_a,  axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c,  axis=0))

ax.plot(times, cond_sig_a.sum(axis=1)[nr_condcodes], 
        '-o', color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.plot(times, cond_sig_c.sum(axis=1)[nr_condcodes], 
        '-o', color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.plot(times, cond_sig_b.sum(axis=1)[nr_condcodes],
        '-o', color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})')
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.set_ylabel('number of DARs')
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))


fig.tight_layout()
output = os.path.join(outfigdir, 'abc_dars_over_time.pdf') 
powerplots.savefig_autodate(fig, output)

plt.show()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(3*4,1*4), sharex=True, sharey=True)

num_uniq_a = np.sum(np.any(cond_sig_a,  axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c,  axis=0))

ax = axs[0]
ax.plot(times, cond_sig_a.sum(axis=1)[nr_condcodes], 
        '-o', color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)
ax.set_ylabel('number of DARs')

ax = axs[1]
ax.plot(times, cond_sig_c.sum(axis=1)[nr_condcodes], 
        '-o', color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

ax = axs[2]
ax.plot(times, cond_sig_b.sum(axis=1)[nr_condcodes],
        '-o', color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})')
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=0)
# ax.set_ylim(ymin=-50, ymax=2000)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(3*4,1*4))

num_uniq_a = np.sum(np.any(cond_sig_a,  axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c,  axis=0))

ax = axs[0]
ax.plot(times, cond_sig_a.sum(axis=1)[nr_condcodes], 
        '-o', color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
ax.plot(dr_times, cond_sig_a.sum(axis=1)[dr_condcodes], 
        '-s', linestyle=(0,(2,1)), color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(dr_times, [0]*len(dr_times),
        '-s', linestyle=(0,(2,1)), color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)
ax.set_ylabel('number of DARs')
# ax.set_ylim(ymin=-500, ymax=12500)

ax = axs[1]
ax.plot(times, cond_sig_c.sum(axis=1)[nr_condcodes], 
        '-o', color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
ax.plot(dr_times, cond_sig_c.sum(axis=1)[dr_condcodes], 
        's', linestyle=(0,(2,1)), color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(dr_times, [0]*len(dr_times),
        's', linestyle=(0,(2,1)), color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-500, ymax=12500)

ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

ax = axs[2]
ax.plot(times, cond_sig_b.sum(axis=1)[nr_condcodes],
        '-o', color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})')
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
ax.plot(dr_times, cond_sig_b.sum(axis=1)[dr_condcodes], 
        's', linestyle=(0,(2,1)), color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})') 
ax.plot(dr_times, [0]*len(dr_times),
        's', linestyle=(0,(2,1)), color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-50, ymax=2000)

ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

fig.tight_layout()
output = os.path.join(outfigdir, 'atac_num_abc_dars.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

# get the full matrix

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# fin3 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_cpm_tensor_250411.npy'
# atac_tensor = np.load(fin3)

# cond_sig_selected = np.any(np.vstack([cond_sig_a, cond_sig_c, cond_sig_b]), axis=0)
# n_dar = cond_sig_selected.sum()
# atac_tensor_sel = atac_tensor[:,:,cond_sig_selected]
# mat = np.log2(1+atac_tensor_sel.reshape(n_cond*n_type, n_dar).T)

# mat.shape